<a href="https://colab.research.google.com/github/seonghunYang/face-recognition/blob/main/%EC%98%81%EC%83%81_%EC%96%BC%EA%B5%B4_%EC%8B%9D%EB%B3%84_%EB%AA%A8%EB%93%88_%EA%B5%AC%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##얼굴 식별 파이프라인
1. face detection
2. face alignment 및 정규화
3. face representation
4. face verification

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

%cd drive/MyDrive/i-mind-face-recognition

In [ ]:
from deepface.commons import functions, distance as dst
from deepface import DeepFace
from retinaface import RetinaFace
import time
import cv2
import matplotlib.pyplot as plt

#face detection

In [ ]:
# 이미지 사진 보여주고, rgb 변환한 값 return
def showImg(img_path):
  img_arr  = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
  plt.figure(figsize=(8, 8))
  plt.imshow(img_arr)
  return img_arr

def drawFrameWithBbox(frame, detected_faces, labels):
  draw_frame = frame.copy()
  idx = 0
  for key in detected_faces.keys():
    face = detected_faces[key]
    face_area = face['facial_area']
    left = face_area[0]
    top = face_area[1]
    right = face_area[2]
    bottom = face_area[3]

    cv2.rectangle(draw_frame, (left, top), (right, bottom), (255, 0, 0), 1)
    cv2.putText(draw_frame, labels[idx], (int(left), int(top - 7)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    landmarks = face['landmarks']
    # landmark 표시
    for key in landmarks:
      cv2.circle(draw_frame, tuple(landmarks[key]), 1, (255, 0, 0), -1)
    idx += 1
  return draw_frame

#face alignment / normalization
1. face crop
2. face alignment
3. face resize and normalization

In [ ]:
from retinaface.commons import postprocess
import numpy as np
from keras.preprocessing import image

In [ ]:
def cropFace(img, area):
  img_copy = img.copy()
  left = area[0]
  top = area[1]
  right = area[2]
  bottom = area[3]
  return img_copy[top: bottom, left: right]

def alignFace(img, landmarks):
  left_eye = landmarks["left_eye"]
  right_eye = landmarks["right_eye"]
  nose = landmarks["nose"]
  mouth_right = landmarks["mouth_right"]
  mouth_left = landmarks["mouth_left"]
  img = postprocess.alignment_procedure(img, right_eye, left_eye, nose)
  return img[:, :, ::-1]

def resizeFace(img, target_size=(112, 112)):
  factor_0 = target_size[0] / img.shape[0]
  factor_1 = target_size[1] / img.shape[1]
  factor = min(factor_0, factor_1)

  dsize = (int(img.shape[1] * factor), int(img.shape[0] * factor))
  img = cv2.resize(img, dsize)

  diff_0 = target_size[0] - img.shape[0]
  diff_1 = target_size[1] - img.shape[1]
  img = np.pad(img, ((diff_0 // 2, diff_0 - diff_0 // 2), (diff_1 // 2, diff_1 - diff_1 // 2), (0, 0)), 'constant')
  
  if img.shape[0:2] != target_size:
    img = cv2.resize(img, target_size)
  
  img_pixels = image.img_to_array(img)
  img_pixels = np.expand_dims(img_pixels, axis = 0)
  img_pixels /= 255 #normalize input in [0, 1]

  return img_pixels

In [ ]:
def processFrame(frame, faces):
  result = []
  for key in faces.keys():
    face = faces[key]
    facial_area = face['facial_area']
    # 얼굴 자르기
    crop_face = cropFace(frame, facial_area)
    # 얼굴 정렬
    landmarks = face['landmarks']
    align_face = alignFace(crop_face, landmarks)
    #resize
    resize_face = resizeFace(align_face)
    result.append(resize_face)
  return result